In [ ]:
import requests
import json
import time
import random
from openpyxl import Workbook
import pandas as pd

In [3]:
# 接口地址
url = "https://m.ctrip.com/restapi/soa2/20591/getGsOnlineResult"

# 请求头（最基本模拟即可）
headers = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    'Origin': 'https://m.ctrip.com',
    'Referer': 'https://you.ctrip.com/',
    'Accept-language': 'zh-CN,zh;q=0.9',
}

# 请求体
payload = {
    "head": {
        "cid": "09031129314942411422",
        "ctok": "",
        "cver": "1.0",
        "lang": "01",
        "sid": "8888",
        "syscode": "09",
        "auth": ""
    },
    "keyword": "博物馆",   # 搜索关键词
    "pageIndex": 1,       # 当前页
    "pageSize": 20,       # 每页数量
    "profile": False,
    "sourceFrom": "",
    "tab": "sight"        # 搜索范围：景点
}

In [ ]:
response = requests.post(url, headers=headers, data=json.dumps(payload),timeout=10)
data = response.json()

# 检查返回的数据结构
print(json.dumps(data, indent=2, ensure_ascii=False))

In [7]:
items = data.get("items", [])

results = []
for item in items:
    results.append({
        "景点名": item.get("word"),
        "英文名": item.get("eName"),
        "城市": item.get("districtName"),
        "URL": item.get("url"),
        "评分": item.get("commentScore"),
        "评论数": item.get("commentCount"),
        # "图片": item.get("imageUrl")
    })

# 打印前几个结果
for r in results[:20]:
    print(r)

{'景点名': '故宫博物院, 北京', '英文名': 'The Palace Museum', '城市': '北京', 'URL': 'https://you.ctrip.com/sight/1/229.html', '评分': 4.8, '评论数': 184740}
{'景点名': '秦始皇帝陵博物院(兵马俑), 西安', '英文名': "Emperor Qinshihuang's Mausoleum Site Museum", '城市': '西安', 'URL': 'https://you.ctrip.com/sight/7/1444.html', '评分': 4.7, '评论数': 126012}
{'景点名': '南京总统府, 南京', '英文名': 'Nanjing Presidential Palace', '城市': '南京', 'URL': 'https://you.ctrip.com/sight/9/1681.html', '评分': 0.0, '评论数': 0}
{'景点名': '上海科技馆, 上海', '英文名': 'Shanghai Science and Technology Museum', '城市': '上海', 'URL': 'https://you.ctrip.com/sight/2/18308.html', '评分': 4.7, '评论数': 40411}
{'景点名': '青岛啤酒博物馆, 青岛', '英文名': 'Tsingtao Beer Museum', '城市': '青岛', 'URL': 'https://you.ctrip.com/sight/5/55144.html', '评分': 4.8, '评论数': 38161}
{'景点名': '杜甫草堂, 成都', '英文名': 'Chengdu Du Fu Thatched Cottage Museum', '城市': '成都', 'URL': 'https://you.ctrip.com/sight/104/4226.html', '评分': 4.7, '评论数': 35446}
{'景点名': '沈阳故宫, 沈阳', '英文名': 'Shenyang Imperial Palace', '城市': '沈阳', 'URL': 'https://you.ctrip.c

### 多城市的爬取

In [ ]:
import pandas as pd
import requests, json, time
import os
from openpyxl import Workbook
ctrip_result = './data'  
class XcSpider(object):
    def __init__(self, city_id, city_name=""):
        self.CityId = str(city_id)
        self.CityName = city_name
        self.url = "https://m.ctrip.com/restapi/soa2/18254/json/getAttractionList"
        self.headers = {
            'content-type': 'application/json',
            'origin': 'https://m.ctrip.com',
            'referer': f'https://m.ctrip.com/webapp/you/gspoi/sight/{self.CityId}.html',
            'accept-language': 'zh-CN,zh;q=0.9',
        }

        # 关键词字典
        self.keywords_dict = {
            "知识科普型": ["博物馆", "科技馆", "动物园", "植物园", "历史文化遗产", "工业遗产", "科研场所", "科研"],
            "自然观赏型": ["国家公园", "自然保护区", "生态景区"],
            "体验考察型": ["农庄", "研学", "生态农业园"],
            "励志拓展型": ["红色教育", "国防教育", "大学"],
            "文化康乐型": ["影视城", "文化小镇"]
        }

        self.wb = Workbook()
        self.ws = self.wb.active
        self.ws.append([
            '大类', '子类关键词', '景点ID', '名称', '评分', '评论数', '区域', '城市', '地址',
            '分类', '等级', '特色', '标签', '门票说明', '是否免费', '图片链接', '详情链接',
            '纬度', '经度'
        ])

    def get_data(self):
        for main_type, subtypes in self.keywords_dict.items():
            for keyword in subtypes:
                print(f"\n 当前任务: {self.CityName}({self.CityId}) - {main_type} - {keyword}")
                self.fetch_keyword(main_type, keyword)

        # 保存原始结果
        raw_file = f'携程景点_{self.CityId}_{self.CityName}_多关键词.xlsx'
        save_path = os.path.join(ctrip_result, raw_file)  # 拼接完整路径
        self.wb.save(save_path)
        print(f'✅ {self.CityName}({self.CityId}) 抓取完成，已保存至 {raw_file}')

        # --- 去重 ---
        print("\n📊 正在去重...")
        data = self.ws.values
        columns = next(data)  
        df = pd.DataFrame(data, columns=columns)

        df_unique = df.drop_duplicates(subset=["景点ID"], keep="first")
        dedup_file = f"携程景点_{self.CityId}_{self.CityName}_多关键词_去重.xlsx"
        save_dir = "ctrip_result"
        dedup_path = os.path.join(save_dir, dedup_file)   # 拼接路径
        df_unique.to_excel(dedup_file, index=False)
        print(f"✅ 去重完成，{self.CityName}({self.CityId}) 结果已保存到 {dedup_file}，剩余 {len(df_unique)} 条")

    def fetch_keyword(self, main_type, keyword):
        page = 1
        while True:
            print(f'正在抓取关键词【{keyword}】第 {page} 页...')
            payload = {
                "index": str(page),
                "count": 20,
                "sortType": 1,
                "districtId": self.CityId,
                "scene": "onlinesearch",
                "keyword": keyword,
                "returnModuleType": "all",
                "filter": {"filterItems": [f"4;{self.CityId}", "0"]},
                "coordinate": {
                    "latitude": 39.99054,
                    "longitude": 116.310171,
                    "coordinateType": "WGS84"
                },
                "head": {
                    "cid": "09031129314942411422",
                    "ctok": "",
                    "cver": "1.0",
                    "lang": "01",
                    "sid": "8888",
                    "syscode": "999",
                    "auth": "",
                    "xsid": "",
                    "extension": []
                }
            }

            try:
                response = requests.post(self.url, headers=self.headers, data=json.dumps(payload), timeout=10)
                data = response.json()

                if "attractionList" in data and data["attractionList"]:
                    self.parse(data, main_type, keyword)
                else:
                    print(f'⚠️ {keyword} 第 {page} 页无数据，结束该关键词。')
                    break

                if len(data["attractionList"]) < payload["count"]:
                    print(f"✅ 关键词【{keyword}】已抓取到最后一页。")
                    break

            except Exception as e:
                print(f'❌ {keyword} 第 {page} 页请求异常: {e}')
                break

            page += 1
            time.sleep(1.5)

    def parse(self, response, main_type, keyword):
        for result in response['attractionList']:
            card = result.get('card', {})
            comment_count = card.get('commentCount', 0)

            if comment_count and comment_count > 0:
                row = [
                    main_type,
                    keyword,
                    card.get('poiId', ''),
                    card.get('poiName', '').strip(),
                    card.get('commentScore', 0),
                    comment_count,
                    card.get('zoneName', ''),
                    card.get('districtName', ''),
                    card.get('displayField', ''),
                    card.get('sightCategoryInfo', ''),
                    card.get('sightLevelStr', ''),
                    ', '.join(card.get('shortFeatures', [])),
                    ', '.join(card.get('tagNameList', [])),
                    card.get('priceTypeDesc', ''),
                    '是' if card.get('isFree', False) else '否',
                    card.get('coverImageUrl', ''),
                    card.get('detailUrl', ''),
                    card.get('coordinate', {}).get('latitude', ''),
                    card.get('coordinate', {}).get('longitude', '')
                ]
                print(f"📍 {row[3]} ({row[2]}) - 评分: {row[4]}, 评论: {row[5]}")
                self.ws.append(row)


In [ ]:
if __name__ == '__main__':
    # 读取城市列表
    cities_df = pd.read_csv("cities_id.csv")  # 假设有 CityId, CityName 列
    for _, row in cities_df.iterrows():
        spider = XcSpider(city_id=row["city_id"], city_name=row.get("city_name", ""))
        spider.get_data()